In [ ]:
from dep_tools.namers import DepItemPath
import folium
from pystac import Item
from odc.stac import load, configure_s3_access

In [ ]:
sensor = "s2"
dataset_id = "geomad"
year = "2023"

bucket_url = "https://dep-public-staging.s3.us-west-2.amazonaws.com/"

In [ ]:
areas = (
    # ("63,20", "Viti_Levu"),
    ("52,22", "Vanuatu"),
    # ("89,16", "Rarotonga"),
    # ("56,49", "Kiribati"),
    # ("75,25", "Samoa"),
    # ("28,32", "PNG")
)

version_1 = "0.3.1"
version_2 = "0.3.2"

configure_s3_access(cloud_defaults=True, aws_unsigned=True)

# for item_id, name in areas:
m = folium.Map(location=[0, 0], zoom_start=2)

item_id, name = areas[0]

for version in [version_2, version_1]:
    zp = not (version == "0.3.0")
    item_path = DepItemPath(sensor, dataset_id, version, time=year, zero_pad_numbers=zp)
    item_url = bucket_url + item_path.stac_path(item_id=item_id)
    item = Item.from_file(item_url)
    data = load([item], bands=["red", "green", "blue"], chunks={})

    data.squeeze().odc.to_rgba(vmin=1000, vmax=4000).odc.add_to(m, name=f"v: {version}")

m.fit_bounds(data.odc.map_bounds())
folium.LayerControl().add_to(m)

m

# m.save(f"comparison_{name}.html")

In [ ]:
item_path = DepItemPath(sensor, dataset_id, "0.3.2", time=year, zero_pad_numbers=True)
item_url = bucket_url + item_path.stac_path(item_id=item_id)
item = Item.from_file(item_url)
data = load([item], bands=["smad", "emad", "bcmad"], chunks={}).squeeze().compute()
data

In [ ]:
m = folium.Map()
m.fit_bounds(data.odc.map_bounds())

for var in data.data_vars:
    data[var].odc.add_to(m, robust=True, name=f"{var}")

folium.LayerControl().add_to(m)

m